In [1]:
import pandas as pd
import xarray as xr

In [120]:
path = "../results/prec_hist_hist_idf_gumbel.nc"
xr.load_dataset(path)

<xarray.Dataset>
Dimensions:      (south_north: 112, west_east: 132, DURACION: 22)
Coordinates:
  * south_north  (south_north) int64 0 1 2 3 4 5 6 ... 106 107 108 109 110 111
  * west_east    (west_east) int64 0 1 2 3 4 5 6 ... 125 126 127 128 129 130 131
  * DURACION     (DURACION) int64 1 2 3 4 5 6 8 10 ... 42 48 60 72 84 96 108 120
    LONGITUD     (south_north, west_east) float32 -100.5 -100.4 ... -97.33
    LATITUD      (south_north, west_east) float32 18.14 18.14 ... 20.63 20.63
Data variables:
    GEV_C        (south_north, west_east, DURACION) float64 -0.774 ... -0.774
    GEV_LOC      (south_north, west_east, DURACION) float64 -5.698e-23 ... -5...
    GEV_SCALE    (south_north, west_east, DURACION) float64 5.136e-23 ... 5.1...
    KTEST_P      (south_north, west_east, DURACION) float64 8.369e-13 ... 8.3...

In [9]:
ds_1 = xr.open_dataset("../temp/prec_hist_hist_days.nc")
ds_1["XLONG"] = ds_1["XLONG"].isel(south_north = 0)
ds_1["XLAT"] = ds_1["XLAT"].isel(west_east = 0)
ds_1 = ds_1.swap_dims( {"west_east": "XLONG", "south_north": "XLAT"} )
ds_2 = xr.open_dataset("../data/CHIRPS_megalopolis.nc")
ds_2 = ds_2.rename_dims( {"time": "XTIME", "longitude": "XLONG",
    "latitude": "XLAT"} ).rename_vars( {"time": "XTIME",
    "longitude": "XLONG", "latitude": "XLAT", "precip": "Pcp"}
    )
ds_1 = ds_1.coarsen( {"XLAT": 2, "XLONG": 2} ).mean()
ds_1.rename_dims( {"XLAT": "south_north", "XLONG": "west_east"}
    ).to_netcdf("../temp/prec_hist_comp_estaciones.nc")
ds_1 = ds_1.interp( {"XLAT": ds_2["XLAT"], "XLONG": ds_2["XLONG"]},
    method = "cubic" ).rename_dims( {"XLAT": "south_north",
    "XLONG": "west_east"} )
ds_1.to_netcdf("../temp/prec_hist_comp_CHIRPS.nc")
ds_1

<xarray.Dataset>
Dimensions:     (XTIME: 11322, bnds: 2, south_north: 50, west_east: 62)
Coordinates:
  * XTIME       (XTIME) datetime64[ns] 1985-01-01T11:30:00 ... 2015-12-31T11:...
    XLAT        (south_north) float32 18.17 18.22 18.27 ... 20.52 20.57 20.62
    XLONG       (west_east) float32 -100.4 -100.4 -100.3 ... -97.43 -97.38
Dimensions without coordinates: bnds, south_north, west_east
Data variables:
    XTIME_bnds  (XTIME, bnds) datetime64[ns] 1985-01-01 ... 2015-12-31T23:00:00
    Pcp         (XTIME, south_north, west_east) float64 nan nan nan ... nan nan
Attributes:
    CDI:          Climate Data Interface version 2.1.0 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    frequency:    day
    history:      Wed Sep 13 09:47:13 2023: cdo daysum data/prec_hist_hist.nc...
    CDO:          Climate Data Operators version 2.1.0 (https://mpimet.mpg.de...